In [40]:
import numpy as np
import pandas as pd
import re

In [41]:
df=pd.read_csv("/Users/sophiegeoghan/Desktop/OB-GYN Webscraping/nysdr_test/NYC_obgyn_final.csv", header=None)
df.columns=["Malpractice","Boro","Education","Name","Specialty"]

In [42]:
def split_out_year(ed):
    try:
        s=re.search(',\n',ed).span()
    except:
        s=(None,None)
    school=ed[0:s[0]]
    year= ed[s[1]:]
    return year
def split_out_school(ed):
    try:
        s=re.search(',\n',ed).span()
    except:
        s=(None,None)
    return ed[0:s[0]]
df['School']=list(map(split_out_school,df.Education))
df['Year']=list(map(split_out_year,df.Education))
df=df.drop("Education",axis=1)

In [43]:
us_states=pd.read_csv("/Users/sophiegeoghan/Desktop/OB-GYN Webscraping/nysdr_test/States.csv")
us_states=us_states[us_states.x!='US']
us_states=us_states['x']

df['State'] = df['School'].str.extract('( [A-Z]{2}$)')
df.State=df.State.str.strip()


In [44]:
df.State=df.State.fillna('outside the US')

In [45]:
private=['Boston University School Of Medicine, Boston Ma', 'Duke University School Of Medicine, Durham Nc',
         'Harvard Medical School, Boston Ma', 'Johns Hopkins University School Of Medicine, Baltimore Md', 
         'New York University School Of Medicine, Ny',
         'Wake Forest University School Of Medicine, Winston-Salem Nc',
         'Weill Cornell Medical College Of Cornell University,New York Ny',
         'Yale University School Of Medicine, New Haven Ct',
         'Vanderbilt University School Of Medicine, Nashville Tn',
        'University Of Wisconsin Medical School, Madison Wi', 'University Of Michigan Medical School, Ann Arbor Mi']
private=list(map(lambda x: x.upper(),private))

In [46]:
# Finding country where they went to school
ind=-1
countries=[]
school_rank=[]
for st in df.State:
    ind+=1
    school=df.loc[ind]["School"]
    if st in us_states.values:
        countries.append('USA')
        if school in private:
            school_rank.append(1)
        else:
            school_rank.append(2)
    else:
        school_rank.append(3)
        try:
            country=re.search(', [A-Z]+$',df.School[ind]).group() #{1}[a-z]
            countries.append(country[2:])#df['School'].str.extract(', [A-Z]{1}[a-z]+$')
        except Exception as e:
            #print(e)
            countries.append('')
            continue
df['Country']=countries
df['School_Rank']=school_rank

In [47]:
def find_num_pay(mal):
    # check Judgments and Arbitration Awards
    start,end =re.search('Number of awards:',mal).span()
    try:
        a= int(mal[end+1:end+2])
    except:
        # check Settlements
        start,end_p =re.search('Number of payments:',mal).span()
        a= int(mal[end_p+1:end_p+2])
    return a

In [48]:
ind=0
num_payments=[]
for m in range(len(df)):
    #print(ind)
    if df.Malpractice[ind]=='None reported':
        num_payments.append(0)
    else:
        a=find_num_pay(df.Malpractice[ind])
        num_payments.append(a)
    ind+=1
df['Num_Payments']=num_payments

In [49]:
df['Name'].apply(lambda x: " ".join(x.title() for x in x.split()))
df['Name']=df['Name'].apply(lambda x: " ".join([x.split()[1].title(),x.split()[0][:-1].title()]))
df['Num_Payments']=df['Num_Payments'].apply(lambda x: int(x))

In [50]:
df.to_csv("nysdr.csv")